## Estimate Cosine Similarity between members and their own party- Grouping all speeches

In [64]:
import pandas as pd
from keras.preprocessing.text import text_to_word_sequence
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from tqdm import tqdm

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
### Load data

In [4]:
path_file = '/Users/cblanesg/cam.blanes Dropbox/Camila Blanes/Congressional-dataMX/data/02-outcomes/01-policy_positioning/01-text_analysis/wordfish/01_lda/speeches_20k.csv'

In [5]:
data = pd.read_csv(path_file).drop('Unnamed: 0', axis = 1)

## group by topic and legislator

In [6]:
input_data = data[['id_legislador', 'legislatura', 'inc_party','clean_speech']]

In [7]:
input_data = input_data.groupby(['id_legislador',
                   'legislatura',
                   'inc_party']).agg(''.join).reset_index()

## Obtain Cosain Similarity between members and median member of party

### Prepare data

In [9]:
coordinadores = pd.read_excel('/Users/cblanesg/cam.blanes Dropbox/Camila Blanes/Congressional-dataMX/data/01-collection_data/00-id_data/04-coordinadores_parlamentarios/coordinadores_parlamentarios.xlsx')
ids_coordinadores = list(coordinadores.id_legislador)

In [10]:
data_members = input_data[~input_data['id_legislador'].isin(ids_coordinadores)]
data_coordinadores = input_data[input_data['id_legislador'].isin(ids_coordinadores)].reset_index().drop('index', axis = 1)

### Cosine Similarity

In [62]:
def party_agenda(data_coordinadores,input_data ):
    all_dataframes = []
    for i in tqdm(range(0, len(data_coordinadores))):
        legislatura = data_coordinadores.loc[i]['legislatura']
        party =  data_coordinadores.loc[i]['inc_party']
        leader =  data_coordinadores.loc[i]['id_legislador']
        
        subset = input_data[input_data['legislatura'] == legislatura][input_data[input_data['legislatura'] == legislatura]['inc_party'] == party]
        vectorizer = TfidfVectorizer()
        response = vectorizer.fit_transform(subset.clean_speech)
        
        tfidf = pd.DataFrame(response.toarray().transpose(),
                   index=vectorizer.get_feature_names())
        cosine_similarity_matrix = np.transpose(np.asmatrix(tfidf))*np.asmatrix(tfidf)
        
        df_cosine = pd.DataFrame(cosine_similarity_matrix)
        temp_cosine = pd.concat([pd.DataFrame({'id_legislador':subset.id_legislador}).reset_index(drop=True), df_cosine], axis=1)
        
        df_temp = pd.DataFrame({'id_legislador':subset.id_legislador})
        index_party_leader  = df_temp.reset_index()[df_temp.reset_index()['id_legislador'] == leader].index[0]
        
        cosine_out = temp_cosine[['id_legislador', index_party_leader]]
        cosine_out.columns = ['id_legislador', 'cosine_similarity']
        cosine_out['party_leader'] = np.where(cosine_out['id_legislador'] == leader, 1, 0)
        cosine_out['legislatura'] = legislatura
        cosine_out['inc_party'] = party
        all_dataframes.append(cosine_out)
    return(pd.concat(all_dataframes))

## Party Agenda Measure

In [65]:
out = party_agenda(data_coordinadores,input_data)

100%|██████████| 37/37 [00:10<00:00,  3.53it/s]


In [69]:
out.to_excel('/Users/cblanesg/cam.blanes Dropbox/Camila Blanes/Congressional-dataMX/data/02-outcomes/01-policy_positioning/01-text_analysis/cosine_similarity/party_agenda_leadership.xlsx')